In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import statistics as st

In [2]:
data = pd.read_csv("loan_train.csv")
test = pd.read_csv('loan_test.csv')
data.head()
# data = pd.concat([data, test], ignore_index=True)
data.shape

(614, 13)

In [3]:
label = LabelEncoder()

data.Self_Employed = data.Self_Employed.fillna('No')
data.Self_Employed.isnull().sum()
data["Self_Employed"] = label.fit_transform(data["Self_Employed"])
print(data.Self_Employed.value_counts())

test.Self_Employed = test.Self_Employed.fillna('No')
test["Self_Employed"] = label.fit_transform(test["Self_Employed"])

# Replacing the NA values as third category C
data.loc[data.Gender.isna(), 'Gender'] = 'C' 
data["Gender"] = label.fit_transform(data["Gender"])
print(data.Gender.value_counts())

test.loc[test.Gender.isna(), 'Gender'] = 'C' 
test["Gender"] = label.fit_transform(test["Gender"])

0    532
1     82
Name: Self_Employed, dtype: int64
2    489
1    112
0     13
Name: Gender, dtype: int64


In [4]:
Married_na_index = data[data['Married'].isna()].index.values
for i in Married_na_index:
    if(data.loc[i, 'Dependents'] == 0):
        data.loc[i, 'Married'] = 'Yes'
    else:
        data.loc[i, 'Married'] = 'No'
        
data.Married.isna().sum()
data.Married = label.fit_transform(data.Married)

Married_na_index = test[test['Married'].isna()].index.values
for i in Married_na_index:
    if(test.loc[i, 'Dependents'] == 0):
        test.loc[i, 'Married'] = 'Yes'
    else:
        test.loc[i, 'Married'] = 'No'
        
test.Married = label.fit_transform(test.Married)

In [5]:
data.Dependents.isna().sum()
data.Dependents = data.Dependents.fillna('0')
data.Dependents = label.fit_transform(data.Dependents)
data.Dependents.value_counts()

test.Dependents = test.Dependents.fillna('0')
test.Dependents = label.fit_transform(test.Dependents)

In [6]:
data.Education = label.fit_transform(data.Education)

test.Education = label.fit_transform(test.Education)

In [7]:
data.Property_Area.isna().sum()
data.Property_Area = label.fit_transform(data.Property_Area)
data.Property_Area.value_counts()

test.Property_Area = label.fit_transform(test.Property_Area)

In [8]:
data.Loan_Status.isna().sum()
data.Loan_Status.value_counts()
data.Loan_Status = data.Loan_Status.apply(lambda x: 1 if x=='Y' else 0)
data.Loan_Status.value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

In [9]:
data.LoanAmount = data.LoanAmount.fillna(data.LoanAmount.mean())
data.Loan_Amount_Term = data.Loan_Amount_Term.fillna(st.mode(data.Loan_Amount_Term))
data.Credit_History = data.Credit_History.fillna(1)

test.LoanAmount = data.LoanAmount.fillna(test.LoanAmount.mean())
test.Loan_Amount_Term = data.Loan_Amount_Term.fillna(st.mode(test.Loan_Amount_Term))
test.Credit_History = data.Credit_History.fillna(1)

In [10]:
test.head()
test_f = test.drop('Loan_ID', axis=1)
test_f

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,2,1,0,0,0,5720,0,146.412162,360.0,1.0,2
1,2,1,1,0,0,3076,1500,128.000000,360.0,1.0,2
2,2,1,2,0,0,5000,1800,66.000000,360.0,1.0,2
3,2,1,2,0,0,2340,2546,120.000000,360.0,1.0,2
4,2,0,0,1,0,3276,0,141.000000,360.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
362,2,1,3,1,1,4009,1777,130.000000,360.0,1.0,2
363,2,1,0,0,0,4158,709,95.000000,300.0,1.0,2
364,2,0,0,0,0,3250,1993,141.000000,360.0,1.0,1
365,2,1,0,0,0,5000,2393,133.000000,360.0,1.0,0


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
y = data.Loan_Status
X = data.drop(['Loan_Status','Loan_ID'], axis=1)

Standardisation = StandardScaler() 
X = Standardisation.fit_transform(X) 
test_f = Standardisation.fit_transform(test_f) 
# print ("\nAfter Standardisation : \n", x_after_Standardisation)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1101)
print(data.Loan_Status.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64


In [13]:
rf = RandomForestClassifier(n_estimators = 5, random_state = 101)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of RandomForestClassifier: {:.3f}'.format(accuracy))

Accuracy of RandomForestClassifier: 0.805


In [14]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 2, max_depth =3)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of DecisionTreeClassifier: {:.3f}'.format(accuracy))

Accuracy of DecisionTreeClassifier: 0.846


In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 123)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of LogisticRegression: {:.3f}'.format(accuracy))

Accuracy of LogisticRegression: 0.846


In [16]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(learning_rate=0.01)
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of XGBoost Classifier: {:.3f}'.format(accuracy))

Accuracy of XGBoost Classifier: 0.846


In [17]:
from sklearn.svm import SVC
sv = SVC(gamma = 'auto')
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Support vector machines: {:.3f}'.format(accuracy))

Accuracy of Support vector machines: 0.846


In [19]:
from sklearn.ensemble import VotingClassifier
estimator = []
estimator.append(('rf', RandomForestClassifier(random_state = 101)))
estimator.append(('dtc', DecisionTreeClassifier(random_state = 2)))
estimator.append(('lr', LogisticRegression(random_state = 123)))
estimator.append(('xgb_clf', XGBClassifier(learning_rate=0.01)))
estimator.append(('sv', SVC(gamma = 'auto', probability = True)))

In [20]:
vot = VotingClassifier(estimators = estimator, voting ='hard') 
vot.fit(X_train, y_train) 
y_pred = vot.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of VotingClassifier: {:.3f}'.format(accuracy))

Accuracy of VotingClassifier: 0.854


In [36]:
pred = vot.predict(test_f)
pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,

In [148]:
pred = pd.DataFrame(pred, columns=['Loan_Status'])
pred = pred.Loan_Status.apply(lambda x:'N' if x==0 else 'Y')
pred = pd.DataFrame(pred)
pred

,Loan_Status
0,Y
1,Y
2,Y
3,Y
4,Y
...,...
362,Y
363,Y
364,Y
365,Y


In [153]:
loan_id = pd.DataFrame(test.Loan_ID, columns=['Loan_ID'])
loan_id
result = pd.concat([loan_id, pred], axis=1)
result = result.set_index('Loan_ID')
result.to_csv('Result.csv')